In [1]:
# !conda activate final_project_env
!mkdir genome
%cd genome
!wget -nc -O genome.fa.gz https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.genomic.fa.gz
!gzip -d -f genome.fa.gz

/data/users/mccallke0364/Final_Project/Sm_Mira_IvT/genome
--2024-11-19 16:25:59--  https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.genomic.fa.gz
Resolving ftp.ebi.ac.uk... 193.62.193.165
Connecting to ftp.ebi.ac.uk|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116797085 (111M) [application/x-gzip]
Saving to: ‘genome.fa.gz’

genome.fa.gz        100%[===================>] 111.39M  26.4MB/s    in 4.9s    

2024-11-19 16:26:05 (22.6 MB/s) - ‘genome.fa.gz’ saved [116797085/116797085]



In [2]:
!wget -O - https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.canonical_geneset.gtf.gz | \
    gzip -f -d > annotations.gtf

--2024-11-19 16:49:11--  https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.canonical_geneset.gtf.gz
Resolving ftp.ebi.ac.uk... 193.62.193.165
Connecting to ftp.ebi.ac.uk|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3228084 (3.1M) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>]   3.08M  3.81MB/s    in 0.8s    

2024-11-19 16:49:12 (3.81 MB/s) - written to stdout [3228084/3228084]



In [ ]:
!samtools faidx genome.fa

In [ ]:
!while IFS= read -r line; do \
    echo "Getting $line from NCBI SRA"; \
    parallel-fastq-dump --sra-id $line --threads 16 --outdir fastq_placenta --gzip; \
    done < sra_list_placenta.txt

##Step 1: Review Metadata and QC Data
1.1 Review QC data from the qc/ folder:
Check the quality of the RNA extraction, library prep, and sequencing process. These details will inform decisions about trimming and filtering reads.
The QC reports may include metrics such as sequencing depth, base composition, GC content, and read length distributions, which can guide trimming and filtering steps.
1.2 Read the reference paper:
Understanding the study in the paper "Winners vs. losers: Schistosoma mansoni intestinal and liver eggs exhibit striking differences in gene expression and immunogenicity" will give you insights into the expected differences between liver and intestine miracidia, helping you interpret the results later.
##Step 2: Read Preprocessing (Trimming and Quality Control)
2.1 Assess the raw reads:
Use tools like FastQC to assess the raw sequencing data. Create a MultiQC report to summarize the results for all samples. This will help identify issues like adapter contamination, GC bias, or low-quality regions.
2.2 Trim low-quality bases and adapters:
Use Trimmomatic or Cutadapt to remove adapters and trim low-quality bases (e.g., Q-score below 20). Focus on trimming the 3' and 5' ends of the reads if necessary.
After trimming, run FastQC again and generate a new MultiQC report to ensure quality has improved.
2.3 Verify trimming results:
Check that the trimmed reads have good quality, proper length distributions, and no obvious contamination. If necessary, adjust the trimming parameters.
##Step 3: Read Alignment to the Reference Genome
3.1 Align trimmed reads:
Use STAR or HISAT2 for aligning the reads to the Schistosoma mansoni genome. The choice of aligner will depend on the quality of your reference genome.
Align the reads in a strand-specific manner since the NEBNext Ultra II Directional mRNA kit was used. This will ensure that the strand information is preserved for downstream analyses.
3.2 Assess alignment quality:
Use SAMtools to check the alignment quality. This will include assessing mapping rate, proper pairs, and other metrics.
You can use Picard tools to generate alignment statistics and visualize any potential issues.
##Step 4: Counting Reads
4.1 Generate gene counts:
Use featureCounts (from the Subread package) or HTSeq-count to generate a gene count matrix based on the aligned BAM files. Ensure you use a GTF file for Schistosoma mansoni gene annotations.
The output should be a count matrix, with genes in rows and samples in columns.
4.2 Inspect counts:
Visualize the count distribution across samples using boxplots or histograms to ensure that the counts look reasonable.
##Step 5: PCA Analysis
5.1 Normalize the counts:
Normalize the gene count data to account for sequencing depth and gene length. Use DESeq2 or edgeR for this purpose.
Normalize to obtain variance-stabilized counts or log-transformed counts for downstream analyses.
5.2 Perform PCA:
Use DESeq2 or prcomp (in R or Python) to perform Principal Component Analysis (PCA) on the normalized count data.
Visualize the PCA results with a ggplot2 scatter plot, coloring by sample origin (liver vs. intestine). Look for clustering patterns between the two groups.
##Step 6: Differential Expression Analysis
6.1 Perform differential expression analysis:
Use DESeq2 to identify differentially expressed genes (DEGs) between the liver and intestine samples.
Set up a contrast in DESeq2 to compare liver versus intestine groups, and run the differential expression pipeline.
6.2 Visualize DEGs:
Use a volcano plot to visualize the DEGs. Plot log-fold changes (logFC) on the x-axis and the significance (–log10 p-value) on the y-axis. Highlight the most significant DEGs.
Consider additional visualizations like MA plots or heatmaps of DEGs, especially genes with high or low expression.
##Step 7: Additional Requirements
7.1 Create a Conda Environment:
Provide an environment.yaml file that includes the required packages for the analysis (e.g., DESeq2, FastQC, Trimmomatic, STAR, featureCounts, MultiQC, etc.).
Ensure the environment is reproducible, and list all dependencies.
7.2 Version Control with Git:
Initialize a Git repository for version control.
Make commits at regular intervals (at least one per week). Write informative commit messages and organize the project structure logically.
7.3 Create a Snakemake Pipeline:
Write a Snakefile to automate the analysis from raw reads to differential expression analysis. The pipeline should handle tasks like trimming, alignment, counting, and DESeq2 analysis in a reproducible manner.
Specify the necessary Conda environments using --use-conda to ensure reproducibility.
##Step 8: Report and Documentation
8.1 Write an IPython Notebook:
Create a well-documented and organized IPython notebook to explain the steps of the analysis.
Include sections for each step of the process, with markdown cells explaining the rationale behind each step and what you are looking for in the output logs and plots.
Include the necessary plots (PCA and volcano plot) and interpret the results.
8.2 Include a MultiQC Report:
Generate a MultiQC report that aggregates the quality control results from all stages of the analysis (raw reads, trimmed reads, alignment, counting, and PCA). This will help assess the overall quality of the data at each stage.
Final Deliverables:
IPython Notebook with full analysis and interpretations.
MultiQC Report with QC information.
environment.yaml file for Conda environment setup.
Gene count matrix.
Snakefile for reproducible analysis pipeline (if aiming for an A grade).
Git repository with version-controlled code (for B or higher).
Proper documentation and clear presentation of results.
By following this structure, you will meet the grade specifications and ensure a reproducible and comprehensive RNA-seq analysis pipeline.



You said:
what does it mean by include YAML that uses --use-conda
ChatGPT said:
ChatGPT
In this context, the requirement to "include YAML that uses --use-conda" refers to specifying the dependencies for your analysis pipeline in a YAML file (specifically for Snakemake), which ensures that the necessary software and tools can be installed and used within a conda environment.

When you're using Snakemake to automate your analysis, you can use the --use-conda option to let Snakemake automatically create and manage conda environments for each of the steps in the workflow. The YAML file, known as the environment file, defines the conda environment's dependencies (i.e., the software packages needed for each step).

Here’s a breakdown:

1. YAML File (Environment Specification)
This YAML file lists all the conda dependencies required for the analysis. You can create an environment YAML file for each tool or step (e.g., FastQC, DESeq2, featureCounts, STAR), and Snakemake will automatically set up the appropriate environments.

Example of a simple environment.yaml file:

yaml
Copy code
name: rnaseq_analysis
channels:
  - bioconda
  - conda-forge
  - defaults
dependencies:
  - python=3.8
  - fastqc
  - star
  - samtools
  - deeq2
  - multiqc
  - r-base
  - r-ggplot2
2. Using --use-conda in Snakemake
When running Snakemake, you specify --use-conda to tell Snakemake to automatically use conda environments for each rule. Snakemake will look for an environment.yaml file in the same directory or specified for each individual step of your workflow and create the environment from the dependencies listed in that file.

Example Snakemake command:

bash
Copy code
snakemake --use-conda
This command will make sure that Snakemake uses conda environments where necessary to run the steps in your pipeline.

3. Snakemake Rules with Environment YAMLs
For each step of your pipeline, you can link to a specific environment YAML file by using the conda directive in your Snakemake rule. For example:

python
Copy code
rule fastqc:
    input:
        "raw_data/{sample}.fastq"
    output:
        "qc/{sample}_fastqc.html"
    conda:
        "envs/fastqc_env.yaml"  # this is your conda environment file for FastQC
    shell:
        "fastqc {input} -o {output}"
Why This is Useful:
Reproducibility: By specifying the environment and the exact versions of dependencies, it ensures that the analysis can be run again with the same software versions, even on a different machine.
Dependency Management: Automatically manages the installation of all required software packages for each step in the workflow without worrying about potential version conflicts.
In summary, to meet the requirement of "include YAML that uses --use-conda," you would:

Create an environment.yaml file listing the necessary software dependencies.
Use Snakemake with the --use-conda option to automatically create conda environments and ensure reproducibility.

in single end seq we get a single end primer that does the seq by synthesis and seq all accross the fragment and reads the fluoresence
for paired end we add an entire second round that adds to the other end and does the reverse end of the original 
star deals with reads that overlap differently based on their number? maybe